In [8]:
import cv2
import os
import glob
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
from models import DnCNN
from math import gamma
from scipy.stats import gennorm
from utils import *

In [9]:
# Please modified the following codes for your own use
modelPath = 'beta2/org/logs/'+'model_29.pth'
f = glob.glob(os.path.join('data', 'Set12', '02.png'))
NoisyPictureName = 'beta10_TDist_Noisy02.png'
CleanPictureName = 'beta10_TDist_Clean02.png'

In [10]:
def getSize(noise):
    k = 1
    for i in range(len(noise.size())):
        k *= noise.size()[i]
    return k

def normalize(data):
    return data/255.

def unnormalize(data):
    return data*255.
def getAlpha(std, beta):
    return np.sqrt(np.square(std)/(gamma(3/beta)/gamma(1/beta)))
def getDenoisedImage(modelPath, f, NoisyPictureName, CleanPictureName, beta = 2):
    ## Translate raw image
    Img = cv2.imread(f[0])
    Img = normalize(np.float32(Img[:,:,0]))
    Img = np.expand_dims(Img, 0)
    Img = np.expand_dims(Img, 1)
    ISource = torch.Tensor(Img)

    ## Add noise
    flatSize = getSize(ISource)
    alpha = getAlpha(25/225.,beta)
    noise = torch.FloatTensor(gennorm.rvs(beta, scale = alpha, size = flatSize, random_state = None))
    noise = noise.view(ISource.size())
    INoisy = ISource + noise
    INoisyTBSave = INoisy

    ###Save Noisy picture
    INoisyTBSave = INoisyTBSave[0,0,:,:]
    INoisyTBSave = unnormalize(np.float32(INoisyTBSave))
    INoisyTBSave = np.expand_dims(INoisyTBSave, 2)
    cv2.imwrite(NoisyPictureName,INoisyTBSave)

    ## Load Model
    net = DnCNN(channels=1, num_of_layers=17)
    device_ids = [0]
    model = nn.DataParallel(net, device_ids=device_ids).cuda()
    model.load_state_dict(torch.load(modelPath))
    model.eval()

    ## send noisy image to model
    ISource, INoisy = Variable(ISource.cuda()), Variable(INoisy.cuda())
    with torch.no_grad(): # this can save much memory
        Out = torch.clamp(INoisy-model(INoisy), 0., 1.)

    ### Save clean picture
    Out = Out[0,0,:,:]
    Out = unnormalize(np.float32(Out.cpu()))
    Out = np.expand_dims(Out, 2)
    cv2.imwrite(CleanPictureName,Out)
    return Out

In [11]:
getDenoisedImage(modelPath, f, NoisyPictureName, CleanPictureName, beta = 2)

array([[[193.9227  ],
        [195.8329  ],
        [195.90233 ],
        ...,
        [195.99063 ],
        [192.4937  ],
        [202.33255 ]],

       [[200.88963 ],
        [199.75377 ],
        [203.22684 ],
        ...,
        [193.62685 ],
        [180.35283 ],
        [191.77849 ]],

       [[196.76936 ],
        [201.41792 ],
        [216.84291 ],
        ...,
        [192.14557 ],
        [180.17471 ],
        [193.42036 ]],

       ...,

       [[183.11514 ],
        [180.10994 ],
        [183.53603 ],
        ...,
        [ 81.014725],
        [101.56604 ],
        [148.68913 ]],

       [[179.89783 ],
        [180.33757 ],
        [179.69196 ],
        ...,
        [ 72.41376 ],
        [115.64726 ],
        [148.24005 ]],

       [[179.18526 ],
        [183.7517  ],
        [182.59023 ],
        ...,
        [ 90.70247 ],
        [134.46954 ],
        [146.34625 ]]], dtype=float32)